Placing Orders and Executing Trades

In [17]:
import pandas as pd
import numpy as np
import tpqoa
from datetime import datetime, timedelta, timezone

In [19]:
# start_time = datetime.now(timezone.utc) - timedelta(days = 5)
# end_time = datetime.now(timezone.utc)

In [21]:
df = pd.read_csv('oanda.cfg')

In [23]:
class ConTrader(tpqoa.tpqoa):

    def __init__(self, config_file, instrument, bar_length, window, units):
        # Initialize with OANDA API
        super().__init__(config_file)
        self.instrument = instrument
        self.bar_length = bar_length
        self.tick_data = pd.DataFrame()
        self.raw_data = pd.DataFrame()
        self.position = 0
        self.units = units

        # Call the method to fetch historical data
        self.get_historical_data()

    def get_historical_data(self):
        """Fetch historical data from OANDA and handle errors properly."""
        # Get the current time in UTC
        start_time = datetime.now(timezone.utc) - timedelta(days=5)
        end_time = datetime.now(timezone.utc)

        try:
            # Fetch historical data
            df = self.get_history(
                instrument=self.instrument,
                start=start_time.isoformat(),
                end=end_time.isoformat(),
                granularity='M1',  # 1-minute bars
                price='M'
            )
        except Exception as e:
            print(f"Error fetching historical data: {e}")
            return  # This will exit the method on error

        # Check if there is an error message in the response
        if 'errorMessage' in df:
            print(f"OANDA API Error: {df['errorMessage']}")
        else:
            print("Data fetched successfully")
            self.tick_data = df.copy()  # Store the data for further use

        # Proceed with any further operations on the df, like renaming columns
        if not self.tick_data.empty:
            self.process_data()

    def process_data(self):
        """Process the fetched historical data."""
        # Perform resampling or any other data operations here
        self.tick_data.rename(columns={"C": self.instrument}, inplace=True)
        self.tick_data = self.tick_data.resample(self.bar_length, label="right").last().dropna().iloc[:-1]

        # Log the processed data if needed
        print("Data processing complete:", self.tick_data)


In [25]:
print(df.head())

                                             [oanda]
0                  account_id = 101-001-29655670-001
1  access_token = 6f38e8a06b6ea8b5898714fcfc7eb78...
2                            account_type = practice


In [27]:
trader = ConTrader("oanda.cfg", "EUR_USD", "H1", window = 1, units = 5000)    

Starting tick data stream...
